In [ ]:
! pip install wrds pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2


In [ ]:
import wrds
import pandas as pd
import numpy as np

# Connect to WRDS
conn = wrds.Connection()

# Query CRSP data (CRSP daily stock file)
query_crsp = """
    SELECT permno, date, vol, prc
    FROM crsp.dsf
    WHERE date >= '2000-01-01'
"""
crsp_data = conn.raw_sql(query_crsp, params=None)

# Close WRDS connection after fetching data
conn.close()

# Convert the 'date' column to proper datetime format
crsp_data['date'] = pd.to_datetime(crsp_data['date'])

# Manually define the inflation factor (CPI value from a base year, e.g., 2013 CPI value)
# Let's assume the CPI value in 2013 is 230.0 (you can adjust this to the actual CPI value from your base year)
base_cpi = 230.0

# Calculate dollar volume (price * volume)
crsp_data['dollar_vol'] = crsp_data['prc'] * crsp_data['vol']

# Inflation-adjusted dollar volume using a fixed inflation factor (CPI 2013)
crsp_data['infl_adj_dollar_vol'] = crsp_data['dollar_vol'] * 0.967  # Example: CPI = 230 in 2013

# Define the function to calculate previous 5-day average and next day's volume
def calculate_avg_and_next_day_vol(df):
    df['avg_past_5'] = df.groupby('permno')['infl_adj_dollar_vol'].transform(
        lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())
    df['next_day_vol'] = df.groupby('permno')['infl_adj_dollar_vol'].shift(-1)
    return df

# Apply the function to calculate the required volumes
crsp_data = calculate_avg_and_next_day_vol(crsp_data)

# Filter the stock-days based on the conditions:
filtered_data = crsp_data[
    (crsp_data['infl_adj_dollar_vol'] > 100000000) &  # More than $100M adjusted
    (crsp_data['infl_adj_dollar_vol'] > 10 * crsp_data['avg_past_5']) &  # 10x the 5-day average
    (crsp_data['infl_adj_dollar_vol'] > 10 * crsp_data['next_day_vol'])  # 10x the next day's volume
]

# Display or export the filtered stock-days
print(filtered_data[['permno', 'date', 'infl_adj_dollar_vol', 'avg_past_5', 'next_day_vol']])

# Optionally, save the filtered results to a CSV
filtered_data.to_csv('filtered_stock_days2000.csv', index=False)


Enter your WRDS username [root]:satyam030400
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: n
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
        permno       date  infl_adj_dollar_vol    avg_past_5  next_day_vol
48949    10051 2022-07-22         3.838224e+08  1.881559e+06  3.134793e+07
65111    10078 2009-04-20         6.907831e+09  1.437793e+08  6.168790e+08
151729   10200 2019-09-20         6.839519e+08  4.659347e+07  4.042424e+07
153039   10201 2011-06-24         1.081063e+08  7.800115e+06  9.596786e+06
194145   10252 2021-11-12         3.609456e+08  3.543162e+07  3.049617e+07
...        ...        ...                  ...           ...           ...
264353   93369 2021-05-06         6.103791e+08  5.641356e+07  5.727236e+07
282967   93376 2011-11-30         1.087363e+08  1.680338e+05  1.005806e+05
285070   93376 2020-04-13         1.074908e+08  2.218274e